# Strip Plot Genome Value Distribution + Fitness in each Objective (2D)

In [ ]:
import pandas as pd
import plotly.graph_objects as go
import numpy as np

def continuously_colored_f1_strip_plot(df:pd.DataFrame, problem_name:str,  min=-1.0, max=1.0):
    '''
    Adjusted version of this stack overflow answer: https://stackoverflow.com/a/67026297/13374784
    '''
    fig = go.Figure()
    N = len(df)
    unique_gene_numbers = df['gene_numbner'].unique()
    for i, gene_number in enumerate(unique_gene_numbers):

        dfs = df.query('gene_numbner == @gene_number')

        if i == 2:#only one marker should gennerate colorbar
            marker=dict(
                size=9,
                color=dfs['f1'],
                colorbar=dict(
                    title='f1',
                ),
                colorscale='Blues',
                cmin=min,
                cmax=max
            )
        else:
            marker = dict(
                size=9,
                color=dfs['f1'],
                colorscale='Blues',
                cmin=min,
                cmax=max
            )
        
        fig.add_trace(go.Scatter(
            y=dfs['gene_values'], 
            x=(i-0.1) + np.random.rand(N) * 0.2,
            mode='markers',
            marker=marker,
            name=str(gene_number),
        ))
    fig.update_layout(showlegend=False, coloraxis_showscale=True, title=problem_name)
    fig.update_xaxes(tickvals=unique_gene_numbers, ticktext=unique_gene_numbers)
    fig.show()

def continuously_colored_f2_strip_plot(df:pd.DataFrame, problem_name:str, min=-1.0, max=1.0):
    '''
    Adjusted version of this stack overflow answer: https://stackoverflow.com/a/67026297/13374784
    '''
    fig = go.Figure()
    N = len(df)
    unique_gene_numbers = df['gene_numbner'].unique()
    for i, gene_number in enumerate(unique_gene_numbers):

        dfs = df.query('gene_numbner == @gene_number')

        if i == 0:#only one marker should gennerate colorbar
            marker=dict(
                size=9,
                color=dfs['f2'],
                colorbar=dict(
                    title='f2',
                ),
                colorscale='Oranges',
                cmin=min,
                cmax=max
            )
        else:
            marker = dict(
                size=9,
                color=dfs['f2'],
                colorscale='Oranges',
                cmin=min,
                cmax=max
            )
        
        fig.add_trace(go.Scatter(
            y=dfs['gene_values'], 
            x=(i-0.1) + np.random.rand(N) * 0.2,
            mode='markers',
            marker=marker,
            name=str(gene_number),
        ))
    fig.update_layout(showlegend=False, coloraxis_showscale=True, title=problem_name)
    fig.update_xaxes(tickvals=unique_gene_numbers, ticktext=unique_gene_numbers)
    fig.show()


def build_df(problem, n_samples = 100):
    df = pd.DataFrame()
    ps = problem._calc_pareto_set(n_pareto_points=n_samples)
    pf = problem.evaluate(ps)

    gene_numbers = np.array( [ np.arange(0,problem.n_var)  for _ in range( n_samples ) ] ).flatten()#TODO: magic number!

    f_1 = np.array( [ pf[ int(ind_number), 0 ] for ind_number in range(n_samples) for _ in range(problem.n_var) ] )
    f_2 = np.array( [ pf[ int(ind_number), 1 ] for ind_number in range(n_samples) for _ in range(problem.n_var) ] )

    df['gene_values'] = ps.flatten()
    df['gene_numbner'] = gene_numbers

    df['f1'] = f_1
    df['f2'] = f_2

    return df

def continuously_colored_strip_plot(problem, problem_name, n_samples=100, min=0.0, max=1.0):
    plot_data_df = build_df(problem, n_samples)
    continuously_colored_f1_strip_plot(plot_data_df, problem_name, min, max)
    continuously_colored_f2_strip_plot(plot_data_df, problem_name, min, max)

In [ ]:
from pymoo_problems.moo.uf import UF1, UF2, UF3

n_var = 10
n_samples = 100
continuously_colored_strip_plot(UF1(n_var=n_var), "UF1", n_samples)
#continuously_colored_strip_plot(UF2(n_var=n_var), "UF2")
#continuously_colored_strip_plot(UF3(n_var=n_var), "UF3")

In [ ]:
from pymoo_problems.moo.zdt import ZDT1, ZDT2, ZDT3

n_var = 4
continuously_colored_strip_plot(ZDT1(n_var=n_var), "ZDT1")
continuously_colored_strip_plot(ZDT2(n_var=n_var), "ZDT2")
continuously_colored_strip_plot(ZDT3(n_var=n_var), "ZDT3")

In [ ]:
from pymoo_problems.moo.maco import MACO

n_var = 4
continuously_colored_strip_plot(MACO(n_var=n_var), "MACO")

# Parallel Coordinate Plot Gene Values + Fitness (color)
This is to better see the connections between the genes.

In [1]:
import pandas as pd
import numpy as np

import plotly.graph_objects as go

def plot_parallel_coordinates(problem, problem_name, n_samples = 20):

        dimensions = ["g"+str(i) for i in range(problem.n_var)]
        objectives = ["f"+str(i+1) for i in range(problem.n_obj)]
        df_labels = dimensions + objectives

        ps = problem._calc_pareto_set(n_pareto_points=n_samples)
        pf = problem.evaluate(ps)
        df_data = np.concatenate((ps, pf), axis=1)
        df = pd.DataFrame(df_data, columns=df_labels)

        fitness_range_f1 = [df["f1"].min(), df["f1"].max()]
        fitness_range_f2 = [df["f2"].min(), df["f2"].max()]

        dimensions = list([ dict(range=[problem.xl[gene_id], problem.xu[gene_id]], values=df[dimensions[gene_id]]) for gene_id in range(problem.n_var) ])

        fig = go.Figure(
                data=go.Parcoords(
                        line=dict(
                                color=df['f1'],
                                colorbar=dict(
                                title='f1',
                                ),
                                colorscale='Blues',
                                cmin = fitness_range_f1[0],
                                cmax = fitness_range_f1[1]
                        ),
                        dimensions = dimensions,
                        name="f1",
                )
        )
        fig.update_layout(title=problem_name+" f1")
        fig.show()
        fig = go.Figure(
                data=go.Parcoords(
                        line=dict(
                                color=df['f2'],
                                colorbar=dict(
                                title='f2',
                                ),
                                colorscale='Oranges',
                                cmin = fitness_range_f2[0],
                                cmax = fitness_range_f2[1]
                        ),
                        dimensions = dimensions,
                        name="f2",
                )
        )
        fig.update_layout(title=problem_name+" f2")
        fig.show()

In [2]:
from pymoo_problems.moo.uf import UF1, UF2, UF3
n_var = 10
n_samples = 100 #TODO: generate the seeds here?
plot_parallel_coordinates( UF1(n_var=n_var), "UF1", n_samples)
plot_parallel_coordinates( UF2(n_var=n_var), "UF2", n_samples)
#plot_parallel_coordinates( UF3(n_var=n_var), "UF3", n_samples)

In [ ]:
from pymoo_problems.moo.uf import UF8, UF9, UF10
n_var = 5
plot_parallel_coordinates( UF8(n_var=n_var), "UF8", n_samples=100)
plot_parallel_coordinates( UF9(n_var=n_var), "UF9", n_samples=100 )
#plot_parallel_coordinates( UF10(n_var=n_var), "UF10", n_samples=100 )

In [ ]:
from pymoo_problems.moo.zdt import ZDT1, ZDT2, ZDT3, ZDT4, ZDT6
n_var = 4
plot_parallel_coordinates( ZDT1(n_var=n_var), "ZDT1" )
plot_parallel_coordinates( ZDT2(n_var=n_var), "ZDT2" )
plot_parallel_coordinates( ZDT3(n_var=n_var), "ZDT3" )

In [ ]:
from pymoo_problems.moo.maco import MACO
n_var = 4
plot_parallel_coordinates( MACO(n_var=n_var), "MACO", 20)


# Parralel Coordinate Plots